In [62]:
%load_ext autoreload
%autoreload 2

import pickle
import pandas as pd
import os
import openai
import numpy as np
import ipdb
import re
from tqdm import tqdm
import matplotlib.pyplot as plt

from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
import spacy
import scipy

nlp = spacy.load("en_core_web_sm")
openai.api_key= os.environ['OPENAI_KEY']

from data_utils import *
from gpt3_utils import *
from eval_utils import *

import copy

pd.set_option('max_rows',500,'max_colwidth',1000)
pd.options.display.float_format = "{:,.2f}".format

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [103]:
data_name = 'bc5cdr_disease'
seed = 42
few_shot_num = 5
number_of_test_samples = 50
selection_strategy = 'random'
sep = ', '
engine='davinci'
filtering = True
logit_bias = 10
sep_logit_bias = 10
new_line_logit_bias = 9

bert_model = 'allenai/scibert_scivocab_uncased'
prompt_filename = '../data/{}_test_prompts_{}_{}.{}.{}.{}'.format(data_name, selection_strategy,seed,few_shot_num,number_of_test_samples,sep)
filename_root = prompt_filename.split('/')[-1]
preds_filename_root = filename_root+'.preds.{}'.format(engine)+'{}-{}-{}'.format(logit_bias, sep_logit_bias, new_line_logit_bias)
if filtering:
    preds_filename_root += '.filter'
preds_filename = '../outputs/'+preds_filename_root
preds_filename += '.knn_selection.{}'.format(bert_model.replace('/','-'))
preds_filename

df = pickle.load(open(preds_filename,'rb'))
df, unfiltered_entities_probs  = recalibrate_positive_entity_threshold(df, 'predictions', 0.4)
df = create_bio_preds(df, 'predictions')
f1, precision, recall = conlleval_eval(df.ner_seq,df.bio_preds)

df = create_bio_preds(df, "predictions")
b_true = [s.replace('I','B') for s in df.ner_seq]
b_pred = [s.replace('I','B') for s in df.bio_preds]

f1, precision, recall = conlleval_eval(b_true,b_pred)

processed 1415 tokens with 60 phrases; found: 41 phrases; correct: 36.
accuracy:  95.90%; (non-O)
accuracy:  95.90%; precision:  87.80%; recall:  60.00%; FB1:  71.29%
                X: precision:  87.80%; recall:  60.00%; FB1:  71.29%  41


,F1,Precision,Recall
0,71.29,87.80,60.00


processed 1415 tokens with 104 phrases; found: 66 phrases; correct: 57.
accuracy:  96.04%; (non-O)
accuracy:  96.04%; precision:  86.36%; recall:  54.81%; FB1:  67.06%
                X: precision:  86.36%; recall:  54.81%; FB1:  67.06%  66


,F1,Precision,Recall
0,67.06,86.36,54.81


In [104]:
scibert_df = df

In [105]:
data_name = 'bc5cdr_disease'
seed = 42
few_shot_num = 5
number_of_test_samples = 50
selection_strategy = 'random'
sep = ', '
engine='davinci'
filtering = True
logit_bias = 10
sep_logit_bias = 10
new_line_logit_bias = 9

bert_model = 'bert-large-uncased'
prompt_filename = '../data/{}_test_prompts_{}_{}.{}.{}.{}'.format(data_name, selection_strategy,seed,few_shot_num,number_of_test_samples,sep)
filename_root = prompt_filename.split('/')[-1]
preds_filename_root = filename_root+'.preds.{}'.format(engine)+'{}-{}-{}'.format(logit_bias, sep_logit_bias, new_line_logit_bias)
if filtering:
    preds_filename_root += '.filter'
preds_filename = '../outputs/'+preds_filename_root
preds_filename += '.knn_selection.{}'.format(bert_model.replace('/','-'))
preds_filename

df = pickle.load(open(preds_filename,'rb'))
df, unfiltered_entities_probs  = recalibrate_positive_entity_threshold(df, 'predictions', 0.4)
df = create_bio_preds(df, 'predictions')
f1, precision, recall = conlleval_eval(df.ner_seq,df.bio_preds)

df = create_bio_preds(df, "predictions")
b_true = [s.replace('I','B') for s in df.ner_seq]
b_pred = [s.replace('I','B') for s in df.bio_preds]

f1, precision, recall = conlleval_eval(b_true,b_pred)

processed 1415 tokens with 60 phrases; found: 37 phrases; correct: 34.
accuracy:  96.47%; (non-O)
accuracy:  96.47%; precision:  91.89%; recall:  56.67%; FB1:  70.10%
                X: precision:  91.89%; recall:  56.67%; FB1:  70.10%  37


,F1,Precision,Recall
0,70.10,91.89,56.67


processed 1415 tokens with 104 phrases; found: 60 phrases; correct: 58.
accuracy:  96.61%; (non-O)
accuracy:  96.61%; precision:  96.67%; recall:  55.77%; FB1:  70.73%
                X: precision:  96.67%; recall:  55.77%; FB1:  70.73%  60


,F1,Precision,Recall
0,70.73,96.67,55.77


In [106]:
bert_df = df

In [107]:
scibert_errors = scibert_df[scibert_df.ner_seq != scibert_df.bio_preds]

In [108]:
bert_errors = bert_df[bert_df.ner_seq != bert_df.bio_preds]

In [109]:
len(scibert_errors.index.intersection(bert_errors.index))/len(scibert_errors)

0.7777777777777778

In [110]:
len(scibert_errors.index.intersection(bert_errors.index)), len(scibert_errors.index.intersection(bert_errors.index))/len(bert_errors)

(14, 0.875)

In [111]:
scibert_errors.loc[scibert_errors.index.intersection(bert_errors.index)]

,sents,ner_seq,bio_preds,entities,num_entities,num_tokens,prompts,empty_prompts,test_ready_prompt,predictions,unfiltered_predictions,gpt3_output_predictions,predictions.post
457,"Following administration of these agents , the presence , and degree of fasciculation were assessed visually on a four point scale by one investigator who was blinded to the drug administered .",O O O O O O O O O O O O B O O O O O O O O O O O O O O O O O O O,O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O,[fasciculation],1,4,"Sentence: Following administration of these agents , the presence , and degree of fasciculation were assessed visually on a four point scale by one investigator who was blinded to the drug administered .\nDiseases: fasciculation","Sentence: Following administration of these agents , the presence , and degree of fasciculation were assessed visually on a four point scale by one investigator who was blinded to the drug administered .\nDiseases:","Sentence: Pain rating was performed using a computerized visual analogue scale (0-100) digitized at 1/s, either immediately online or one hour or one day after injection.\nDiseases: pain\n\nSentence: Injections areas were assessed for the presence of bruising at 48 and 72 hours after each injection.\nDiseases: bruising\n\nSentence: Fasciculations in six areas of the body were scored from 0 to 3 and summated as a total fasciculation score.\nDiseases: fasciculations, fasciculation\n\nSentence: In the evening of postoperative day 1, patients were evaluated for TNSs by a physician unaware of the drug administered and the details of the anesthetic procedure.\nDiseases: tnss\n\nSentence: The intensity of headache and nausea was measured with a 100-mm visual analog scale.\nDiseases: headache, nausea\n\nSentence: Following administration of these agents , the presence , and degree of fasciculation were assessed visually on a four point scale by one investigator who was blinded to the drug ...",[],[fasciculation],"({'id': 'cmpl-4Js6xIw9EQzaxqglnLXXIMO2a4FZn', 'object': 'text_completion', 'created': 1640641583, 'model': 'davinci:2020-05-03', 'choices': [{ ""finish_reason"": ""stop"", ""index"": 0, ""logprobs"": { ""text_offset"": [ 1005, 1010, 1012, 1019, 1019, 1019, 1019, 1019 ], ""token_logprobs"": [ -0.0017959059, -4.406191e-05, -0.000368853, -0.5330265, -6.718934e-05, -0.521621, -0.9267341, -0.26457965 ], ""tokens"": [ "" fasc"", ""ic"", ""ulation"", ""\n"", ""\n"", ""Following"", "" the"", "" administration"" ], ""top_logprobs"": [ { "" fasc"": -0.0017959059 }, { ""ic"": -4.406191e-05 }, { ""ulation"": -0.000368853 }, { ""\n"": -0.5330265 }, { ""\n"": -6.718934e-05 }, { ""Following"": -0.521621 }, { "" the"": -0.9267341 ...",[]
3911,Peroxisomes in hepatomas and hyperplastic preneoplastic liver lesions induced in mice by 500 ppm alpha - benzene hexachloride were examined histochemically and electron microscopically .,O O B O O O B I O O O O O O O O O O O O O O O O O,O O B O O O O O O O O O O O O O O O O O O O O O O,"[liver lesions, hepatomas]",2,6,"Sentence: Peroxisomes in hepatomas and hyperplastic preneoplastic liver lesions induced in mice by 500 ppm alpha - benzene hexachloride were examined histochemically and electron microscopically .\nDiseases: liver lesions, hepatomas",Sentence: Peroxisomes in hepatomas and hyperplastic preneoplastic liver lesions induced in mice by 500 ppm alpha - benzene hexachloride were examined histochemically and electron microscopically .\nDiseases:,"Sentence: Heparan sulphate-associated anionic sites in the glomerular basement membrane were studied in rats 8 months after induction of diabetes by streptozotocin and in age-adn sex-matched control rats, employing the cationic dye cuprolinic blue.\nDiseases: diabetes\n\nSentence: To clarify the effects of bromocriptine on prolactinoma cells in vivo, immunohistochemical, ultrastructural and morphometrical analyses were applied to estrogen-induced rat prolactinoma cells 1 h and 6 h after injection of br

In [112]:
scibert_errors[['sents','entities','predictions','unfiltered_predictions']].merge(bert_errors[['sents','predictions','unfiltered_predictions']],on='sents')

,sents,entities,predictions_x,unfiltered_predictions_x,predictions_y,unfiltered_predictions_y
0,"Following administration of these agents , the presence , and degree of fasciculation were assessed visually on a four point scale by one investigator who was blinded to the drug administered .",[fasciculation],[],[fasciculation],[],[fasciculation]
1,Peroxisomes in hepatomas and hyperplastic preneoplastic liver lesions induced in mice by 500 ppm alpha - benzene hexachloride were examined histochemically and electron microscopically .,"[liver lesions, hepatomas]",[hepatomas],[hepatomas],[],[liver]
2,The differential predictive value of levodopa responsiveness for the outcome of parkinsonian ' off ' signs and LID and the different correlations of ventral lesion volume with dyskinesias and parkinsonian ' off ' signs indicate that different anatomical or pathophysiological substrates may be responsible for the generation of parkinsonian ' off ' signs and dyskinesias .,"[parkinsonian, lid, dyskinesias]","[dyskinesias, parkinsonian, parkinsonian ' off ' signs]","[dyskinesias, parkinsonian, parkinsonian ' off ' signs, ventral lesion volume]",[dyskinesias],[dyskinesias]
3,"CCNU was used most commonly in the treatment of lymphoma , mast cell tumour , brain tumour , histiocytic tumours and epitheliotropic lymphoma .","[lymphoma, mast cell tumour, epitheliotropic lymphoma, brain tumour, histiocytic tumours]","[lymphoma, mast cell tumour, brain tumour]","[lymphoma, mast cell tumour, brain tumour, histiocytic tumours, epitheliotropic lymphoma]","[lymphoma, mast cell tumour, brain tumour]","[lymphoma, mast cell tumour, brain tumour, histiocytic tumours, epitheliotropic lymphoma]"
4,The symptoms of methamphetamine ( METH ) - induced psychosis are similar to those of paranoid type schizophrenia .,"[psychosis, paranoid type schizophrenia]","[ psychosis, schizophrenia]","[methamphetamine, psychosis, schizophrenia]",[psychosis],"[psychosis, methamphetamine]"
5,"RESULTS : Verapamil caused AF promotion in six dogs , increasing mean duration of AF induced by burst pacing , from 8 + / - 4 s ( mean + / - S . E . ) to 95 + / - 39 s ( P < 0 . 01 vs . control ) at a loading dose of 0 . 1 mg / kg and 228 + / - 101 s ( P < 0 . 0005 vs . control ) at a dose of 0 . 2 mg / kg .",[af],[],"[af promotion, burst pacing, loading dose, dose]",[],[af]
6,"RESULTS : Of 60 epileptic women with periconceptional folic acid ( 0 . 8 mg ) - containing multivitamin supplementation , no one developed epilepsy - related side effects during the periconception period .","[epileptic, epilepsy]","[epilepsy, side effects]","[epilepsy, periconceptional folic acid, side effects]",[epilepsy],"[epilepsy, epilepsy related side effects]"
7,"The latter was characterized by a pronounced increase in protein excretion , especially proteins with molecular weight greater than 40 , 000 Da .",[protein excretion],[],[],[],[protein excretion]
8,Neuroleptic malignant syndrome is thought to be a result of dopamine D2 receptor blockade in the striatum of the basal ganglia .,[neuroleptic malignant syndrome],[ basal ganglia],"[dopamine d2 receptor blockade, striatum, basal ganglia]",[ basal ganglia],"[dopamine, basal ganglia, receptor blockade]"
9,"Calcitriol therapy suppresses serum levels of parathyroid hormone ( PTH ) in patients with renal failure but has several drawbacks , including hypercalcemia and / or marked suppression of bone turnover , which may lead to adynamic bone disease .","[renal failure, suppression of bone turnover, adynamic bone disease, hypercalcemia]",[hypercalcemia],"[hypercalcemia, adynamic bone disease]",[hypercalcemia],"[hypercalcemia, adynamic bone disease]"


In [65]:
query_word = 'hypokalemia'
query_vector = None

if query_word in nlp.vocab:
    query_vector = nlp.vocab[query_word].vector

In [82]:
nlp = spacy.load("en_core_sci_lg")
sims = nlp.vocab.vectors.most_similar(np.array([nlp.vocab['hypokalemia'].vector]),n=10)

In [100]:
[nlp.vocab[sim].text for sim in sims[0][0]]

['hypokalemia',
 'sitagliptin-induced',
 'hyperkalemia',
 'hypernatremia/hyperchloremia',
 'hypomagnesemia',
 'hyponatremia',
 'hypopotassemia',
 'foscarnet-induced',
 'hypernatremia',
 'hypo-hypernatremia']

In [113]:
seed_stats = pd.read_csv('bc5cdr_disease.post_filtering_prompt_results.tsv')

In [171]:
all_seeds = []

for seed in seed_stats.p:
    all_seeds.extend([s.replace("'","") for s in seed[1:-1].replace('\n','').split("' '")])

In [172]:
all_seeds = set(all_seeds)

In [173]:
all_seeds = list(all_seeds)

In [174]:
seed_vecs = [np.mean([nlp.vocab[w.text].vector if w.text in nlp.vocab else np.zeros(nlp.vocab.vectors.shape[1]) for w in nlp(seed)],axis=0) for seed in all_seeds]

In [182]:
sort = np.argsort([np.array(nlp.vocab['basal ganglia'].vector).dot(seed) for seed in seed_vecs])[::-1]

In [183]:
np.array(all_seeds)[sort]

array(['antihyperalgesic activity', 'mtdna damage', 'convulsions',
       'visual disturbance', 'purpura', 'dexamethasone',
       'partial loss of quadriceps functions', 'phlebitis', 'myelopathy',
       'petechiae', 'outcome', 'nausea', 'potassium', 'meth mouth',
       'glioblastoma', 'mitochondrial abnormalities',
       'ergotamine / caffeine', 'visual task', 'blood pressure',
       'hoe 140', 'oxidative phosphorylation', 'thrombosis',
       'sumatriptan'], dtype='<U36')

In [181]:
seed_stats

,Unnamed: 0,seed,Avg Yes Prob for Positive Phrases,Avg Yes Prob for Negative Phrases,Prob Difference (Pos-Neg),Min Yes Prob in Pos Phrases,Max Yes Prob in Neg Phrases,0.5 F1,0.5 Prec,0.5 Recall,0.4 F1,0.4 Prec,0.4 Recall,0.4 Accuracy,Prompt,full_prompt,p,ranking_score
0,0,42,0.44,0.29,0.15,0.36,0.37,0.14,1.00,0.08,0.76,1.00,0.62,0.83,['potassium' 'hoe 140' 'visual task' 'nausea'\n 'partial loss of quadriceps functions' 'visual disturbance'],"Does the phrase ""potassium"" refer to a disease?\nNo\n\nDoes the phrase ""hoe 140"" refer to a disease?\nNo\n\nDoes the phrase ""visual disturbance"" refer to a disease?\nYes\n\nDoes the phrase ""visual task"" refer to a disease?\nNo\n\nDoes the phrase ""partial loss of quadriceps functions"" refer to a disease?\nYes\n\nDoes the phrase ""nausea"" refer to a disease?\nYes",['potassium' 'hoe 140' 'visual task' 'nausea'\n 'partial loss of quadriceps functions' 'visual disturbance'],0.91
1,1,0,0.47,0.27,0.20,0.35,0.38,0.47,1.00,0.31,0.92,1.00,0.85,0.93,['ergotamine / caffeine' 'mtdna damage' 'sumatriptan' 'meth mouth'\n 'mitochondrial abnormalities' 'glioblastoma'],"Does the phrase ""ergotamine / caffeine"" refer to a disease?\nNo\n\nDoes the phrase ""mtdna damage"" refer to a disease?\nNo\n\nDoes the phrase ""glioblastoma"" refer to a disease?\nYes\n\nDoes the phrase ""sumatriptan"" refer to a disease?\nNo\n\nDoes the phrase ""mitochondrial abnormalities"" refer to a disease?\nYes\n\nDoes the phrase ""meth mouth"" refer to a disease?\nYes",['ergotamine / caffeine' 'mtdna damage' 'sumatriptan' 'meth mouth'\n 'mitochondrial abnormalities' 'glioblastoma'],1.11
2,2,14,0.37,0.20,0.17,0.24,0.26,0.00,0.00,0.00,0.63,1.00,0.46,0.76,['dexamethasone' 'blood pressure' 'antihyperalgesic activity' 'phlebitis'\n 'convulsions' 'petechiae'],"Does the phrase ""dexamethasone"" refer to a disease?\nNo\n\nDoes the phrase ""blood pressure"" refer to a disease?\nNo\n\nDoes the phrase ""petechiae"" refer to a disease?\nYes\n\nDoes the phrase ""antihyperalgesic activity"" refer to a disease?\nNo\n\nDoes the phrase ""convulsions"" refer to a disease?\nYes\n\nDoes the phrase ""phlebitis"" refer to a disease?\nYes",['dexamethasone' 'blood pressure' 'antihyperalgesic activity' 'phlebitis'\n 'convulsions' 'petechiae'],0.80
3,3,21,0.46,0.27,0.19,0.32,0.40,0.38,1.00,0.23,0.88,0.92,0.85,0.90,['outcome' 'ergotamine / caffeine' 'antihyperalgesic activity'\n 'myelopathy' 'convulsions' 'petechiae'],"Does the phrase ""outcome"" refer to a disease?\nNo\n\nDoes the phrase ""ergotamine / caffeine"" refer to a disease?\nNo\n\nDoes the phrase ""petechiae"" refer to a disease?\nYes\n\nDoes the phrase ""antihyperalgesic activity"" refer to a disease?\nNo\n\nDoes the phrase ""convulsions"" refer to a disease?\nYes\n\nDoes the phrase ""myelopathy"" refer to a disease?\nYes",['outcome' 'ergotamine / caffeine' 'antihyperalgesic activity'\n 'myelopathy' 'convulsions' 'petechiae'],1.07
4,4,33,0.51,0.25,0.27,0.39,0.37,0.70,1.00,0.54,0.92,1.00,0.85,0.93,['potassium' 'mtdna damage' 'oxidative phosphorylation' 'nausea' 'purpura'\n 'thrombosis'],"Does the phrase ""potassium"" refer to a disease?\nNo\n\nDoes the phrase ""mtdna damage"" refer to a disease?\nNo\n\nDoes the phrase ""thrombosis"" refer to a disease?\nYes\n\nDoes the phrase ""oxidative phosphorylation"" refer to a disease?\nNo\n\nDoes the phrase ""purpura"" refer to a disease?\nYes\n\nDoes the phrase ""nausea"" refer to a disease?\nYes",['potassium' 'mtdna damage' 'oxidative phosphorylation' 'nausea' 'purpura'\n 'thrombosis'],1.18
